#  Jogo variante do puzzle 2084

## Introdução à Inteligência Artificial edição 2021/22
### Projeto nº 2

<img src=".\imagens_2048\1024s.PNG" alt="Drawing" style="width: 200px;"/>

### Grupo: 48

#### Elementos do Grupo

Nome: Afonso Esteves

Número: 54394

Nome: João Anjos

Número: 54476

Nome: Vicente Sousa

Número: 55386

## Introdução
Este é um esqueleto do relatório que podem naturalmente expandir, colocando mais células de texto e de código.

## Formulação do Jogo 2048 em termos de estados e de operadores

### Descrição da representação dos estados do jogo

A representação de estados é conseguida através da definição da classe Jogo2048State, filha de GameState. Esta classe define as funções básicas para a manipulação e avaliação de estados. A sua implementação é a seguinte:

In [ ]:
from jogos import *
import copy

ref = [0,0,0,0]

def alignLeft(str, chars):
    return (" "*(chars-len(str)))+str

def reverse(line):
    return line[::-1]

def transpose(matrix):
    return list(map(lambda x: list(x), zip(*matrix)))

def removeZeros(line):
    return list(filter( lambda x: x != 0, line))

def sumStep(line):
    if len(line) < 2:
        return False

    done = False
    for i in range(len(line)-2, -1, -1):
        if line[i] == line[i+1]:
            line[i+1] *= 2
            line[i] = 0
            i -= 1
            done = True
    return done

def pad(line):
    return ref[:len(ref)-len(line):] + line

def sumLine(line):
    li = removeZeros(line)

    while sumStep(li):
        li = removeZeros(li)
    return pad(li)

actions = {
    "direita": lambda m: list( map( lambda l: sumLine(l), m )), 
    "esquerda": lambda m: list( map( lambda l: reverse(sumLine(reverse(l))) , m )),
    "cima": lambda m: transpose( map( lambda l: reverse(sumLine(reverse(l))) , transpose(m) ) ),
    "baixo": lambda m: transpose( map( lambda l: sumLine(l) , transpose(m) ) ),
}


class Jogo2048State(GameState):
    
    """Returns a new state representing the board after the attacker player chooses a direction"""
    def __collapse(self, direction):
        try:
            newstate = Jogo2048State(to_move="defensor", utility=0, board = actions[direction](self.board), moves=self.moves+1)
        except KeyError:
            raise RuntimeError("Error - invalid direction of movement" )

        return newstate
    
    """Returns a new state representing the board after a player action, doesn't check whether the action is valid or not"""
    def next_move(self, move):
        if self.to_move == "atacante":
            return self.__collapse(move)
        elif self.to_move == "defensor":
            newstate = Jogo2048State(to_move="atacante", utility=0, board = copy.deepcopy(self.board), moves=self.moves+1)
            newstate.board[int(move[0])][int(move[2])] = 2
            return newstate
        else:
            raise RuntimeError("Error - invalid player descriptor")

    """Returns the other player, the one not playing this turn"""
    def other(self):
        if self.to_move == "atacante":
            return "defensor"
        if self.to_move == "defensor":
            return "atacante"
        else:
            raise RuntimeError("Error - invalid player descriptor")

    """Prints the board"""
    def display(self):
        print("="*28)
        for i in self.board:
            for j in i:
                print(alignLeft(str(j), 5), end=" ")
            print()
        print("="*28)

    """Returns all valid moves for the state"""
    def get_moves(self):
        if self.to_move == "atacante":
            return [ a for a in ["cima", "direita", "baixo", "esquerda"] if self.__collapse(a).board != self.board ]
        if self.to_move == "defensor":
            res = []
            for i in range(3, -1, -1):
                for j in range(4):
                    if self.board[i][j] == 0:
                        res.append(str(i)+","+str(j))
            return res
        else:
            raise RuntimeError("Error - invalid player descriptor")

### Testes da formulação

#### Situações iniciais dos jogos
Uso do construtor e "display" de jogos iniciais

Construção de um novo jogo com a situação inicial seguinte:

<img src=".\imagens_2048\inicial_2048.PNG" alt="Drawing" style="width: 150px;"/>
<p style="text-align: center;">Figura 1</p>

In [ ]:
class Jogo2048_48(Game):

    def __init__(self, pos1, pos2):
        self.initial = Jogo2048State(to_move = "atacante", utility = 0, board = [ [ 0 for j in range(4)] for i in range(4)], moves = 0)
        self.initial.board[pos1[0]][pos1[1]]=2
        self.initial.board[pos2[0]][pos2[1]]=2
        self.points = 0

    def actions(self, state):
        return state.get_moves()

    def result(self, state, move):
        ref = state.next_move(move)
        ref._replace(utility = self.utility(ref, ref.to_move))
        return ref

    def utility(self, state, player):
        #TODO
        return

    def terminal_test(self, state):
        return not (state.to_move == "defensor" or bool(len(state.get_moves())))

    def to_move(self, state):
        return state.to_move

    def display(self, state):
        state.display()

    def jogar(self, jogador1, jogador2, verbose=True):
        #TODO
        return super().jogar(jogador1, jogador2, verbose)

game = Jogo2048_48((3,2), (3,3))


Eis o display desse estado inicial do jogo:

In [ ]:
game.display(game.initial)

#### As acções
Demonstração das acções possíveis e execução de acções para diversas situações que podem ser simples, por exemplo, iniciais ou mais complexas (a meio do jogo, já com muitas células preechidas.

As acções para o estado inicial do jogo na Fig. 1

In [ ]:
initial_acts = game.actions(game.initial)

#### Execução das acções

Execute a acção direita para o estado da figura 1 e faça o seu display

In [ ]:
state2 = game.result(game.initial, "direita")
game.display(state2)

Mostre o estado inicial outra vez, para confirmar que não se "deformou" devido à execução da acção. 

In [ ]:
game.display(game.initial)

Mostre como ficaria a execução da acção na situação na figura 2

<img src=".\imagens_2048\deslizar_2048.PNG" alt="Drawing" style="width: 350px;"/>
<p style="text-align: center;">Figura 2</p>

In [ ]:
fig2 = [[0,2,4,2],[2,0,8,2],[2,16,8,2],[2048,32,8,2]]
state3 = game.result(Jogo2048State(to_move = "atacante", utility= 2138, board = fig2), "baixo")
game.display(state3)#remove after testing

Mostre que ao aplicar a sequência de acções, (criada na célula a seguir) ao estado inicial da Figura 3, ficará com o jogo na situação ilustrada na Figura 4. O ideal é fazer uma função que executa uma sequência de acções.

| <p style="text-align: center;">Início</p>      | <p style="text-align: center;">Fim</p> |
| :---        |    :----:   |
| <img src="imagens_2048\seq_teste_begin.PNG" alt="Drawing" style="width: 150px;"/>      | <img src="imagens_2048\seq_teste_end.PNG" alt="Drawing" style="width: 150px;"/>       |   |
| <p style="text-align: center;">Figura 3</p>   | <p style="text-align: center;">Figura 4</p>        |

In [1]:
seqTeste=['cima',"1,0","cima","3,3","cima","2,0","direita","2,0","esquerda","1,0",
          "baixo","0,2","baixo","0,0","direita","2,0","cima","2,2","baixo","1,0",
          "esquerda","0,1","baixo","0,2","esquerda","0,2","baixo","0,3","cima","2,3",
          "cima","2,0","esquerda","3,2","cima","1,2","esquerda","3,1","direita","3,1",
          "direita","0,0"]

fig3 = [[0,0,0,0],[0,0,0,0],[0,2,2,0],[0,0,0,0]]
state4 = game.resultActions(Jogo2048State(to_move = "atacante", utility = 0, board = fig3), seqTeste)
game.display(state4)#remove after testing

#### Demonstração que o teste de estado final do jogo funciona
Podem testar para a situação na Figura 4.

<img src=".\imagens_2048\the-end-2084.PNG" alt="Drawing" style="width: 150px;"/>
<p style="text-align: center;">Figura 4</p>

In [ ]:
fig4 = [[16,128,32,16],[8,32,2,8],[2,4,16,2],[4,2,8,4]]
print(str(game.terminal_test(Jogo2048State(to_move="atacante",utility=0,board=fig4))))#remove print after testing

## Jogos entre jogadores simples
Nesta secção irão realizar alguns jogos, para verificar a modelização

### Jogadores aleatórios de ataque e de defesa
Mostre que a sua função que realiza jogos entre dois jogadores (com e sem timeout) funciona para dois jogadores aleatórios.  

Faça o display de um dos jogos realizados atrás

### Jogadores de ataque e de defesa obsessivos


Crie um dois jogadores obsessivos: o atacante obsessivo, prefere sempre por esta ordem, as acções: 'cima', 'esquerda','direita' e 'baixo'. O obsessivo da defesa prefere a célula mais no topo e mais à esquerda possível.
Realize um jogo entre eles e faça o seu display.

## Exemplos de jogadores alfabeta
 Descreva e teste nesta secção as várias funções de avaliação desenvolvidas tanto para o ataque como para a defesa.

## Exemplos de jogos entre alguns desses jogadores e o Hipólito

## Exemplos de jogos entre dois dos vários jogadores desenvolvidos

## Processo de selecção dos jogadores para o torneio
Descreva o processo de selecção dos jogadores campeões, para entrar no campeonato "todos contra todos".

<img src=".\imagens_2048\ormwphixo7y21.jpg" alt="Drawing" style="width: 350px;"/>


*...aqui*